In [10]:
import requests
import pandas as pd
import numpy as np
import time
import random
from bs4 import BeautifulSoup
import multiprocessing as mp
from threading import Thread
import concurrent

In [11]:
data = pd.read_csv(r'C:\Users\9Suser\Desktop\模型訓練資料\tbrain_train_final_0603.csv')[['hyperlink', 'content']]

In [12]:
data = data[0:5]

In [13]:
def number_p(soup, firstwords, endwords, link):    
    flag = False
    content = []
    
    if('mops' in link):
        tags = soup.find_all('font', class_=None, text=True, attrs={'data-reactid':None})
    elif('domestic.judicial' in link):   #司法院公告
        tags = soup.find_all('pre', class_=None, text=True, attrs={'data-reactid':None})
    else:
        tags = soup.find_all('p', class_=None, text=True, attrs={'data-reactid':None})
   
    
    if firstwords not in str(tags):
        for tag in tags:
            content.append(tag.contents[0])
            if endwords in tag.contents[0]:
                break
        
    else:
        for tag in tags:

            if firstwords in tag.contents[0]:
                flag = True

            if flag:
                content.append(tag.contents[0])
                if endwords in tag.contents[0]:
                    break

    return content

In [14]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'}

In [15]:
def crawl(data):
    for index, single_data in data.iterrows():
        print(index)
        if(('tvbs' not in single_data['hyperlink'])):
            try:

                if(('mingpao' in single_data['hyperlink']) | ('coolloud' in single_data['hyperlink'])):
                    response = requests.get(single_data['hyperlink'], headers=headers)    
                    response.encoding = 'utf-8'
                    content = response.text

                else:  
                    response = requests.get(single_data['hyperlink'], headers=headers)
                    content = response.text

                if('mirrormedia' in single_data['hyperlink']):
                    soup = BeautifulSoup(content.replace('<br>', '/n'), 'html.parser')
                else:
                    soup = BeautifulSoup(content, 'html.parser')

                split_words = single_data['content'].split('###')
                firstwords = split_words[0][-6:-1]
                endwords = split_words[-1][1:6]
                all_content = number_p(soup, firstwords=firstwords, endwords=endwords, link=single_data['hyperlink'])
                single_data['content'] = all_content

            except:
                single_data['content'] = ''
                print(single_data['hyperlink'])
        else:
            single_data['content'] = ''
            print(single_data['hyperlink'])

        if index % 10 == 0:
            time.sleep(random.random())
        time.sleep(0.1)

<module 'pymp' from 'C:\\Users\\9Suser\\Anaconda3\\envs\\pyscraper\\lib\\site-packages\\pymp\\__init__.py'>

In [16]:
import pymp
#ex_array = pymp.shared.array((100,), dtype='uint8')
with pymp.Parallel(3) as p:
    crawl(data)
        

AttributeError: module 'pymp' has no attribute 'Parallel'

In [8]:
a = [crawl(index, single_data) for index, single_data in data.iterrows()]
workers = []
for i in range(3):
    worker = Thread(target=a)
    worker.start()
    #workers.append(worker)
for worker in workers:
    worker.join()

0
1
2
3
4


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\9Suser\Anaconda3\envs\pyscraper\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\9Suser\Anaconda3\envs\pyscraper\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'list' object is not callable

Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\9Suser\Anaconda3\envs\pyscraper\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\9Suser\Anaconda3\envs\pyscraper\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'list' object is not callable
Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\9Suser\Anaconda3\envs\pyscraper\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\9Suser\Anaconda3\envs\pyscraper\lib\threading.py", line 864, in run
    self._target(*self._args, **se

In [ ]:
data

In [ ]:
data.to_csv(r'C:\Users\9Suser\Desktop\模型訓練資料\tbrain_train_final_0603_crawl2.csv')